# Séance 2 : compléments sur API

**Open Data Soft** a développé un outil de gestion des données ouvertes, qui est de plus en plus utilisé, en particulier par la mairie de Paris. Vous pouvez trouver l'aide en ligne à cette adresse : 

<https://help.opendatasoft.com/apis/ods-explore-v2/explore_v2.1.html>

L'intérêt d'interroger l'API directement est que les opérations sur la base seront faites par le serveur, et non pas sur votre machine. Ce qui est très intéressant, en particulier dans le cas de travail en temps réel (création d'une appli mobile sur la base d'une API par exemple ou d'un tableau de bord temps réel) ou de travail sur des données très importantes (plusieurs giga octets, voire plus).

**ATTENTION** : Comme vous pourrez le voir, les requêtes sont envoyées sous la forme d'une chaîne de caractères. Il faut donc faire usage des `""` pour la chaîne globale, et des `''` pour les chaînes passées en paramètre dans cette chaîne globale (ou l'inverse).

## API Vélib

Nous allons voir l'utilisation de cet outil en interrogeant l'API Velib, proposée par la mairie de Paris. Toutes les informations sont disponibles à cette adresse :

<https://opendata.paris.fr/explore/dataset/velib-disponibilite-en-temps-reel/api/>

Comme vous le verrez, on utilise les mêmes concepts qu'en SQL.

### Premier essai

L'interrogation se fait, sous Python, avec la fonction `get()` de la librairie `requests` (normalement déjà présente). Le résultat peut être transformé en `JSON` (puis en dictionnaire sous Python) grâce à la fonction `json()` sur celui-ci.

On obtient un objet ayant deux composantes (si tout va bien) :

- `total_counts` : le nombre d'éléments correspondant à la requête (ici, le nombre de stations) ;
- `results` : un tableau contenant une partie des résultats.

Par défaut, la limite est de 10 éléments.

In [1]:
import requests

url_base = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records"
resultat = requests.get(url_base).json()
resultat

{'total_count': 1484,
 'results': [{'stationcode': '44015',
   'name': "Rouget de L'isle - Watteau",
   'is_installed': 'OUI',
   'capacity': 20,
   'numdocksavailable': 8,
   'numbikesavailable': 11,
   'mechanical': 0,
   'ebike': 11,
   'is_renting': 'OUI',
   'is_returning': 'OUI',
   'duedate': '2024-12-04T13:18:53+00:00',
   'coordonnees_geo': {'lon': 2.3963020229163, 'lat': 48.778192750803},
   'nom_arrondissement_communes': 'Vitry-sur-Seine',
   'code_insee_commune': '94081'},
  {'stationcode': '32017',
   'name': 'Basilique',
   'is_installed': 'OUI',
   'capacity': 22,
   'numdocksavailable': 5,
   'numbikesavailable': 16,
   'mechanical': 14,
   'ebike': 2,
   'is_renting': 'OUI',
   'is_returning': 'OUI',
   'duedate': '2024-12-04T13:19:50+00:00',
   'coordonnees_geo': {'lon': 2.3588666820200914, 'lat': 48.93626891059109},
   'nom_arrondissement_communes': 'Saint-Denis',
   'code_insee_commune': '93066'},
  {'stationcode': '13007',
   'name': 'Le Brun - Gobelins',
   'is_in

#### Et si on essaie de tout récupérer

La première idée est de mettre le paramètre `limit` au maximum de ce qu'on doit obtenir comme résultat, pour tout récupérer d'un coup.

Comme vous pouvez le voir ci-dessous, il n'est pas possible de demander plus de 100 élèments en une fois (sauf certaines opérations que l'on verra plus loin).

In [2]:
requests.get(url_base + "?limit=" + str(resultat["total_count"])).json()

{'error_code': 'InvalidRESTParameterError',
 'message': 'Invalid value for limit API parameter: 1484 was found but -1 <= limit <= 100 is expected.'}

#### Comment tout obtenir alors ?

Pour récupérer tous les éléments dans un tel cas, il est nécessaire de faire une boucle et d'utiliser le paramètre `offset` qui permet de passer les premiers résultats.

Le code ci-dessous permet donc de récupérer toutes les stations dans la liste `resultat_final`.

In [3]:
resultat_final = []
for i in range(0, resultat["total_count"], 100):
    temp = requests.get(url_base + "?limit=100&offset=" + str(i)).json()
    resultat_final += temp["results"]

# on trouve bien nos 1484 résultats
len(resultat_final)

1484

#### Que faire des résultats ?

Pour nos besoins ultérieurs (en analyse, calculs, graphiques...), il est intéressant de voir qu'on peut transformer ce résultat `JSON` en un dataframe `pandas` très facilement (cf ci-dessous).

In [4]:
import pandas

data = pandas.DataFrame(resultat_final)
data

,stationcode,name,is_installed,capacity,numdocksavailable,numbikesavailable,mechanical,ebike,is_renting,is_returning,duedate,coordonnees_geo,nom_arrondissement_communes,code_insee_commune
0,44015,Rouget de L'isle - Watteau,OUI,20,8,11,0,11,OUI,OUI,2024-12-04T13:18:53+00:00,"{'lon': 2.3963020229163, 'lat': 48.778192750803}",Vitry-sur-Seine,94081
1,32017,Basilique,OUI,22,5,16,14,2,OUI,OUI,2024-12-04T13:19:50+00:00,"{'lon': 2.3588666820200914, 'lat': 48.93626891...",Saint-Denis,93066
2,13007,Le Brun - Gobelins,OUI,48,41,7,3,4,OUI,OUI,2024-12-04T13:14:31+00:00,"{'lon': 2.3534681351338, 'lat': 48.835092787824}",Paris,75056
3,6003,Saint-Sulpice,OUI,21,0,20,16,4,OUI,OUI,2024-12-04T13:20:32+00:00,"{'lon': 2.3308077827095985, 'lat': 48.85165383...",Paris,75056
4,7002,Vaneau - Sèvres,OUI,35,9,25,14,11,OUI,OUI,2024-12-04T13:19:52+00:00,"{'lon': 2.3204218259346, 'lat': 48.848563233059}",Paris,75056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,15056,Place Balard,OUI,22,2,32,26,6,OUI,OUI,2024-12-04T13:18:50+00:00,"{'lon': 2.2784192115068, 'lat': 48.836395736424}",Paris,75056
1480,8052,Balzac - Champs Elysées,OUI,30,11,19,11,8,OUI,OUI,2024-12-04T13:20:07+00:00,"{'lon': 2.3001953959465, 'lat': 48.872699639621}",Paris,75056
1481,40002,Bleuets - Bordières,NON,1,1,0,0,0,NON,NON,2024-08-27T09:20:23+00:00,"{'lon': 2.4543687905029, 'lat': 48.802117531472}",Créteil,94028
1482,4005,Quai des Célestins - Henri IV,OUI,14,3,10,6,4,OUI,OUI,2024-12-04T13:17:19+00:00,"{'lon': 2.3624535, 'lat': 48.8512971}",Paris,75056


En analysant ce résultat, on peut déjà voir qu'on a des stations en double, voire en triple.

In [5]:
data.groupby("stationcode").size().sort_values(ascending=False)

stationcode
20201    2
15029    2
48006    2
14002    2
14010    2
        ..
17126    1
17125    1
17124    1
17123    1
92007    1
Length: 1320, dtype: int64

Voici les différentes variables récupérées pour chaque station.

In [6]:
data.columns

Index(['stationcode', 'name', 'is_installed', 'capacity', 'numdocksavailable',
       'numbikesavailable', 'mechanical', 'ebike', 'is_renting',
       'is_returning', 'duedate', 'coordonnees_geo',
       'nom_arrondissement_communes', 'code_insee_commune'],
      dtype='object')

### Restriction et projection

Premières étapes classiques de l'interrogation d'une base de données, quelque soit son format (relationnel, NoSQL, autre), la *restriction* (sélection d'élèments sur la base d'une condition) et la *projection* (sélection de certaines variables) sont bien évidemment possibles avec cet outil.


#### Projection 
Ici, on ne sélectionne que les codes des stations (`stationcode`) et le nom de celles-ci (`name`).

In [7]:
requests.get(url_base + "?select=stationcode,name").json()

{'total_count': 1484,
 'results': [{'stationcode': '44015', 'name': "Rouget de L'isle - Watteau"},
  {'stationcode': '32017', 'name': 'Basilique'},
  {'stationcode': '13007', 'name': 'Le Brun - Gobelins'},
  {'stationcode': '6003', 'name': 'Saint-Sulpice'},
  {'stationcode': '5110', 'name': 'Lacépède - Monge'},
  {'stationcode': '40011', 'name': 'Bas du Mont-Mesly'},
  {'stationcode': '21010', 'name': 'Silly - Galliéni'},
  {'stationcode': '30002', 'name': 'Jean Rostand - Paul Vaillant Couturier'},
  {'stationcode': '7003', 'name': 'Square Boucicaut'},
  {'stationcode': '17041', 'name': 'Guersant - Gouvion-Saint-Cyr'}]}

#### Restriction

On peut donc aussi faire une restriction avec la clause `where` (on voit qu'on est très proche du langage SQL).

On cherche ici les stations avec une capacité d'accueil supérieure à 30.

In [8]:
requests.get(url_base + "?select=stationcode,capacity&where=capacity>30").json()

{'total_count': 632,
 'results': [{'stationcode': '13007', 'capacity': 48},
  {'stationcode': '30002', 'capacity': 40},
  {'stationcode': '7003', 'capacity': 60},
  {'stationcode': '17041', 'capacity': 36},
  {'stationcode': '11025', 'capacity': 43},
  {'stationcode': '15047', 'capacity': 52},
  {'stationcode': '17026', 'capacity': 40},
  {'stationcode': '8050', 'capacity': 45},
  {'stationcode': '13101', 'capacity': 34},
  {'stationcode': '31024', 'capacity': 38}]}

#### Avec tri du résultat

Toujours dans la même idée de faire les mêmes opérations qu'en SQL, on peut trier le résultat avec la clause `order_by`.

Même résultat que précédemment, mais trié par ordre croissant de la capacité.

In [9]:
requests.get(url_base + "?select=stationcode,capacity&where=capacity>30&order_by=capacity").json()

{'total_count': 632,
 'results': [{'stationcode': '21323', 'capacity': 31},
  {'stationcode': '5123', 'capacity': 31},
  {'stationcode': '8055', 'capacity': 31},
  {'stationcode': '20019', 'capacity': 31},
  {'stationcode': '17016', 'capacity': 31},
  {'stationcode': '8105', 'capacity': 31},
  {'stationcode': '21004', 'capacity': 31},
  {'stationcode': '20117', 'capacity': 31},
  {'stationcode': '19124', 'capacity': 31},
  {'stationcode': '7024', 'capacity': 31}]}

Et ce tri peut bien évidemment être décroissant en ajoutant `DESC` après le critère de tri.

Toujours même résultat, mais trié par ordre décroissant de la capacité.

In [10]:
requests.get(url_base + "?select=stationcode,capacity&where=capacity>30&order_by=capacity DESC").json()

{'total_count': 632,
 'results': [{'stationcode': '7009', 'capacity': 76},
  {'stationcode': '15030', 'capacity': 74},
  {'stationcode': '15028', 'capacity': 71},
  {'stationcode': '5034', 'capacity': 69},
  {'stationcode': '16025', 'capacity': 68},
  {'stationcode': '12106', 'capacity': 68},
  {'stationcode': '12157', 'capacity': 68},
  {'stationcode': '32012', 'capacity': 68},
  {'stationcode': '1013', 'capacity': 67},
  {'stationcode': '19045', 'capacity': 67}]}

#### Combinaisons de conditions

On peut bien évidemment combiner plusieurs conditions avec les différents opérateurs logiques : `AND`, `OR` et `NOT`.

Nous avons ici les stations avec une capacité supérieure à 20, qui ne sont pas en état de laisser la possibilité de louer les vélos.

In [11]:
requests.get(url_base + "?select=stationcode,capacity,is_renting&where=capacity>20 AND is_renting='NON'&limit=100").json()

{'total_count': 17,
 'results': [{'stationcode': '18203', 'capacity': 40, 'is_renting': 'NON'},
  {'stationcode': '43003', 'capacity': 26, 'is_renting': 'NON'},
  {'stationcode': '42207', 'capacity': 37, 'is_renting': 'NON'},
  {'stationcode': '10162', 'capacity': 24, 'is_renting': 'NON'},
  {'stationcode': '3005', 'capacity': 33, 'is_renting': 'NON'},
  {'stationcode': '18010', 'capacity': 26, 'is_renting': 'NON'},
  {'stationcode': '40014', 'capacity': 28, 'is_renting': 'NON'},
  {'stationcode': '40015', 'capacity': 24, 'is_renting': 'NON'},
  {'stationcode': '40004', 'capacity': 25, 'is_renting': 'NON'},
  {'stationcode': '40001', 'capacity': 28, 'is_renting': 'NON'},
  {'stationcode': '44010', 'capacity': 30, 'is_renting': 'NON'},
  {'stationcode': '15063', 'capacity': 23, 'is_renting': 'NON'},
  {'stationcode': '13048', 'capacity': 28, 'is_renting': 'NON'},
  {'stationcode': '18027', 'capacity': 30, 'is_renting': 'NON'},
  {'stationcode': '40009', 'capacity': 30, 'is_renting': 'NO

#### Recherche dans une chaîne de caractères

On a trois fonctions qui permettent de chercher un chaîne (ou sous-chaîne) de caractères dans une variable (ou même globalement en indiquant `*`), ces premières étant sensibles à la casse :

- `startswidth(champs, 'chaîne')` : cherche les éléments pour lesquels le champs cité débute par la chaîne
- `suggest(champs, 'chaîne')`: cherche les éléments pour lesquels le champs cité contient la chaîne
- `search(champs, 'chaîne')` : cherche les éléments pour lesquels le champs cité est exactement égal à la chaîne

On veut toutes les stations dont le nom commence par "Exelmans".

In [12]:
requests.get(url_base + "?select=stationcode,name&where=startswith(name,'Exelmans')").json()

{'total_count': 2,
 'results': [{'stationcode': '16040', 'name': 'Exelmans - Michel-Ange'},
  {'stationcode': '16039', 'name': 'Exelmans - Versailles'}]}

On veut toutes les stations dont le nom contient "Versailles".

In [13]:
requests.get(url_base + "?select=stationcode,name&where=suggest(name,'Versailles')").json()

{'total_count': 3,
 'results': [{'stationcode': '16039', 'name': 'Exelmans - Versailles'},
  {'stationcode': '16041', 'name': 'Versailles - Claude Terrasse'},
  {'stationcode': '15203', 'name': 'Porte de Versailles'}]}

On cherche précisément la station "Exelmans - Versailles".

In [14]:
requests.get(url_base + "?select=stationcode,name&where=search(name,'Exelmans - Versailles')").json()

{'total_count': 1,
 'results': [{'stationcode': '16039', 'name': 'Exelmans - Versailles'}]}

#### Recherche dans une liste

On dispose de l'opérateur `IN (val1, val2, ...)` permettant de tester si un champs a une valeur comprise dans la liste passée à la suite.

On souhaite obtenir les stations dans les villes de Clichy et Colombes.

In [15]:
# clause IN
requests.get(url_base + "?select=stationcode,nom_arrondissement_communes&where=nom_arrondissement_communes IN ('Clichy','Colombes')").json()

{'total_count': 18,
 'results': [{'stationcode': '27005',
   'nom_arrondissement_communes': 'Colombes'},
  {'stationcode': '27001', 'nom_arrondissement_communes': 'Colombes'},
  {'stationcode': '27002', 'nom_arrondissement_communes': 'Colombes'},
  {'stationcode': '27006', 'nom_arrondissement_communes': 'Colombes'},
  {'stationcode': '27004', 'nom_arrondissement_communes': 'Colombes'},
  {'stationcode': '21104', 'nom_arrondissement_communes': 'Clichy'},
  {'stationcode': '21111', 'nom_arrondissement_communes': 'Clichy'},
  {'stationcode': '21120', 'nom_arrondissement_communes': 'Clichy'},
  {'stationcode': '21118', 'nom_arrondissement_communes': 'Clichy'},
  {'stationcode': '21119', 'nom_arrondissement_communes': 'Clichy'}]}

#### Distance à un objet géométrique

Un des points cruciaux actuellement est la géo-localisation, en particulier dans le cadre d'une appli mobile. On peut ainsi requêter la base de données en cherchant, dans ce premier exemple, les éléments pour lesquels la distance entre un point géographique (stocké dans un champs) et un objet géométrique est inférieure à une certaine distance passée en paramètre.

Ici, nous cherchons les stations à moins de 800m de l'IUT Paris-Rives de Seine. Il faut noter que l'ordre des coordonnées (ici longitude et latitude) dépend de la façon dont elles sont stockées dans le champs. 

In [16]:
requests.get(url_base + "?select=stationcode,name&where=within_distance(coordonnees_geo, geom'POINT(2.267888940737877 48.84197963193564)', 800m)").json()

{'total_count': 15,
 'results': [{'stationcode': '15068',
   'name': 'Général Martial Valin - Pont du Garigliano'},
  {'stationcode': '16118', 'name': 'Michel-Ange - Parent de Rosan'},
  {'stationcode': '16040', 'name': 'Exelmans - Michel-Ange'},
  {'stationcode': '16041', 'name': 'Versailles - Claude Terrasse'},
  {'stationcode': '16037', 'name': 'Molitor - Michel-Ange'},
  {'stationcode': '16039', 'name': 'Exelmans - Versailles'},
  {'stationcode': '15104', 'name': 'Hôpital Européen Georges Pompidou'},
  {'stationcode': '16038', 'name': 'Molitor - Chardon-Lagache'},
  {'stationcode': '16032', 'name': "Eglise d'Auteuil"},
  {'stationcode': '15059', 'name': 'Parc André Citroën'}]}

En complément, on peut en plus récupérer la distance calculée entre ces coordonnées et un objet géométrique.

In [17]:
requests.get(url_base + "?select=stationcode,name,distance(coordonnees_geo, geom'POINT(2.267888940737877 48.84197963193564)') as distance&where=within_distance(coordonnees_geo, geom'POINT(2.267888940737877 48.84197963193564)', 800m)").json()

{'total_count': 15,
 'results': [{'stationcode': '15068',
   'name': 'Général Martial Valin - Pont du Garigliano',
   'distance': 458.73543527109746},
  {'stationcode': '16118',
   'name': 'Michel-Ange - Parent de Rosan',
   'distance': 710.8022353937183},
  {'stationcode': '16040',
   'name': 'Exelmans - Michel-Ange',
   'distance': 606.4513544801562},
  {'stationcode': '16041',
   'name': 'Versailles - Claude Terrasse',
   'distance': 355.48687528576613},
  {'stationcode': '16037',
   'name': 'Molitor - Michel-Ange',
   'distance': 552.7017995780018},
  {'stationcode': '16039',
   'name': 'Exelmans - Versailles',
   'distance': 279.35537451504047},
  {'stationcode': '15104',
   'name': 'Hôpital Européen Georges Pompidou',
   'distance': 725.9808199551429},
  {'stationcode': '16038',
   'name': 'Molitor - Chardon-Lagache',
   'distance': 393.6530345752737},
  {'stationcode': '16032',
   'name': "Eglise d'Auteuil",
   'distance': 615.2029327979726},
  {'stationcode': '15059',
   'name'

#### Comparaison à une zone géographique

La fonction `in_bbox()` teste si un champs est inclus dans une zone géographique rectangulaire, délimitée par deux points.

Ici, on récupère les stations dans un rectangle contenant l'IUT.

In [18]:
requests.get(url_base + "?select=stationcode,name&where=in_bbox(coordonnees_geo,48.84,2.26,48.85,2.27)").json()

{'total_count': 8,
 'results': [{'stationcode': '16041', 'name': 'Versailles - Claude Terrasse'},
  {'stationcode': '16037', 'name': 'Molitor - Michel-Ange'},
  {'stationcode': '16116', 'name': 'George Sand - Jean de La Fontaine'},
  {'stationcode': '16039', 'name': 'Exelmans - Versailles'},
  {'stationcode': '16038', 'name': 'Molitor - Chardon-Lagache'},
  {'stationcode': '16034', 'name': "Porte d'Auteuil"},
  {'stationcode': '16032', 'name': "Eglise d'Auteuil"},
  {'stationcode': '16033', 'name': "Marché d'Auteuil"}]}

Et ici, celles qui n'y sont pas donc.

In [19]:
requests.get(url_base + "?select=stationcode,name&where=not(in_bbox(coordonnees_geo,48.84,2.26,48.85,2.27))").json()

{'total_count': 1476,
 'results': [{'stationcode': '44015', 'name': "Rouget de L'isle - Watteau"},
  {'stationcode': '32017', 'name': 'Basilique'},
  {'stationcode': '13007', 'name': 'Le Brun - Gobelins'},
  {'stationcode': '6003', 'name': 'Saint-Sulpice'},
  {'stationcode': '5110', 'name': 'Lacépède - Monge'},
  {'stationcode': '40011', 'name': 'Bas du Mont-Mesly'},
  {'stationcode': '21010', 'name': 'Silly - Galliéni'},
  {'stationcode': '30002', 'name': 'Jean Rostand - Paul Vaillant Couturier'},
  {'stationcode': '7003', 'name': 'Square Boucicaut'},
  {'stationcode': '17041', 'name': 'Guersant - Gouvion-Saint-Cyr'}]}

A noter qu'il existe une fonction permettant de chercher si un point appartient à une zone géographique de type polygone.

### Calcul d'agrégat

On peut aussi demander à l'API de faire un certain nombre de claculs en amont, en particulier des calculs d'agrégats, avec en plus la clause `group_by` qui permet de faire ce cacul pour chaque modalité d'un champs.

Ici, on a le nombre de stations par ville et la capacité totale de celles-ci.

In [20]:
requests.get(url_base + "?select=nom_arrondissement_communes,count(*),sum(capacity)&group_by=nom_arrondissement_communes").json()

{'results': [{'nom_arrondissement_communes': 'Alfortville',
   'count(*)': 5,
   'sum(capacity)': 122},
  {'nom_arrondissement_communes': 'Arcueil',
   'count(*)': 4,
   'sum(capacity)': 118},
  {'nom_arrondissement_communes': 'Argenteuil',
   'count(*)': 7,
   'sum(capacity)': 225},
  {'nom_arrondissement_communes': 'Asnières-sur-Seine',
   'count(*)': 13,
   'sum(capacity)': 329},
  {'nom_arrondissement_communes': 'Aubervilliers',
   'count(*)': 13,
   'sum(capacity)': 404},
  {'nom_arrondissement_communes': 'Bagneux',
   'count(*)': 6,
   'sum(capacity)': 155},
  {'nom_arrondissement_communes': 'Bagnolet',
   'count(*)': 8,
   'sum(capacity)': 249},
  {'nom_arrondissement_communes': 'Bobigny',
   'count(*)': 5,
   'sum(capacity)': 132},
  {'nom_arrondissement_communes': 'Bois-Colombes',
   'count(*)': 2,
   'sum(capacity)': 60},
  {'nom_arrondissement_communes': 'Boulogne-Billancourt',
   'count(*)': 29,
   'sum(capacity)': 877},
  {'nom_arrondissement_communes': 'Bourg-la-Reine',
 

On peut bien évidemment ordonner ce résultat pour avoir les villes ayant le plus de stations en premier.

In [21]:
requests.get(url_base + "?select=nom_arrondissement_communes,count(*) as nb_stations,sum(capacity)&group_by=nom_arrondissement_communes&order_by=nb_stations DESC").json()

{'results': [{'nom_arrondissement_communes': 'Paris',
   'nb_stations': 991,
   'sum(capacity)': 31893},
  {'nom_arrondissement_communes': 'Boulogne-Billancourt',
   'nb_stations': 29,
   'sum(capacity)': 877},
  {'nom_arrondissement_communes': 'Montreuil',
   'nb_stations': 23,
   'sum(capacity)': 787},
  {'nom_arrondissement_communes': 'Issy-les-Moulineaux',
   'nb_stations': 22,
   'sum(capacity)': 728},
  {'nom_arrondissement_communes': 'Pantin',
   'nb_stations': 21,
   'sum(capacity)': 565},
  {'nom_arrondissement_communes': 'Saint-Denis',
   'nb_stations': 19,
   'sum(capacity)': 666},
  {'nom_arrondissement_communes': 'Ivry-sur-Seine',
   'nb_stations': 18,
   'sum(capacity)': 590},
  {'nom_arrondissement_communes': 'Vitry-sur-Seine',
   'nb_stations': 16,
   'sum(capacity)': 425},
  {'nom_arrondissement_communes': 'Créteil',
   'nb_stations': 14,
   'sum(capacity)': 218},
  {'nom_arrondissement_communes': 'Asnières-sur-Seine',
   'nb_stations': 13,
   'sum(capacity)': 329},
  

### Fonctions spéciales de découpage

#### Découpage d'un champs en intervalles

La fonction `RANGE()` permet de transformer un champs contenant une valeur quantitative en plusieurs modalités, en créant des intervalles sur ces valeurs.

Elle a deux fonctionnement possible :

- par intervalles de même taille, en passant en paramètre un seul entier, qui sera la taille des intervalles créés
- par intervalles définies explicitement, en passant en paramètre la liste des bornes des intervalles

On répartit ici les stations sur la base de leur capacité, en créant des intervalles de taille 15.

In [22]:
requests.get(url_base + "?select=count(*)&group_by=RANGE(capacity,15)").json()

{'results': [{'RANGE(capacity,15)': '[0, 14]', 'count(*)': 43},
  {'RANGE(capacity,15)': '[15, 29]', 'count(*)': 716},
  {'RANGE(capacity,15)': '[30, 44]', 'count(*)': 509},
  {'RANGE(capacity,15)': '[45, 59]', 'count(*)': 166},
  {'RANGE(capacity,15)': '[60, 74]', 'count(*)': 49},
  {'RANGE(capacity,15)': '[75, 89]', 'count(*)': 1}]}

Ici, on créé nous-même les intervalles (du minimum -- avec `*` -- à 14, de 15 à 19, de 20 à 25, de 25 à 30, de 30 à 40, et de 40 au maximum -- avec `*` encore). On renomme aussi le résultat avec la clause `as`.

In [23]:
requests.get(url_base + "?select=count(*)&group_by=RANGE(capacity,*,15,20,25,30,40,*) as capacity").json()

{'results': [{'capacity': '[*, 14]', 'count(*)': 43},
  {'capacity': '[15, 19]', 'count(*)': 135},
  {'capacity': '[20, 24]', 'count(*)': 303},
  {'capacity': '[25, 29]', 'count(*)': 278},
  {'capacity': '[30, 39]', 'count(*)': 407},
  {'capacity': '[40, *]', 'count(*)': 318}]}

Il faut noter que ce découpage peut aussi se faire sur une date, en se basant sur une unité pour le découpage (chaque jour, chaque année, chaque heure...).

#### Découpage selon un niveau de zoom

La fonction `GEO_CLUSTER()` permet de répartir les éléments, géo-localisés par un champs spécifié, en fonction d'un niveau de zoom défini (entre 1 et 25).

Nous découpons ici les stations sur la base d'un niveau de zoom égal à 10, ce qui créé 14 clusters de stations. On récupère de plus les centres de ces clusters.

In [24]:
requests.get(url_base + "?select=count(*)&group_by=GEO_CLUSTER(coordonnees_geo,10)").json()

{'results': [{'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.87072540889531,
     'lon': 2.3640824202600053}},
   'count(*)': 613},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.8389606385267,
     'lon': 2.414170871167604}},
   'count(*)': 174},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.890181980981694,
     'lon': 2.308858638720178}},
   'count(*)': 205},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.912563062357634,
     'lon': 2.2608207606456503}},
   'count(*)': 33},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.842625451678245,
     'lon': 2.257794298724655}},
   'count(*)': 136},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.947931905942305,
     'lon': 2.243483570803489}},
   'count(*)': 7},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.87852969300002,
     'lon': 2.194462900981307}},
   'count(*)': 15},
  {'GEO_

## TRAVAIL A FAIRE

### Données

A partir de l'API **Observation météorologique historiques France (SYNOP)**, vous devez répondre aux demandes qui suivent. Vous trouverez des informations sur cette base à cette adresse :

<https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/information/?sort=date>

Le code ci-dessous permet de récupérer les premiers éléments.

In [25]:
url_base = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/donnees-synop-essentielles-omm/records"
requests.get(url_base).json()


{'total_count': 2531637,
 'results': [{'numer_sta': '78925',
   'date': '2017-04-06T15:00:00+00:00',
   'pmer': 101830,
   'tend': -10.0,
   'cod_tend': '8',
   'dd': 110,
   'ff': 8.3,
   't': 303.45,
   'td': 294.25,
   'u': 58,
   'vv': 53550.0,
   'ww': None,
   'w1': None,
   'w2': None,
   'n': None,
   'nbas': None,
   'hbas': None,
   'cl': None,
   'cm': None,
   'ch': None,
   'pres': 101740.0,
   'niv_bar': None,
   'geop': None,
   'tend24': -10.0,
   'tn12': None,
   'tn24': None,
   'tx12': None,
   'tx24': None,
   'tminsol': None,
   'sw': None,
   'tw': None,
   'raf10': None,
   'rafper': None,
   'per': None,
   'etat_sol': None,
   'ht_neige': None,
   'ssfrai': None,
   'perssfrai': None,
   'rr1': 0.0,
   'rr3': 0.0,
   'rr6': 0.0,
   'rr12': -0.1,
   'rr24': -0.1,
   'phenspe1': None,
   'phenspe2': None,
   'phenspe3': None,
   'phenspe4': None,
   'nnuage1': None,
   'ctype1': None,
   'hnuage1': None,
   'nnuage2': None,
   'ctype2': None,
   'hnuage2': None,


### Demandes

- Récupérer les 500 premières observations
- Récupérer les observations faites en 2020
- Récupérer les observations à Ajaccio
- Récupérer les observations faites à plus de 200 mètres d'altitude, en 2022


In [26]:
pandas.DataFrame(requests.get(url_base).json()["results"])

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,altitude,libgeo,codegeo,nom_epci,code_epci,nom_dept,code_dep,nom_reg,code_reg,mois_de_l_annee
0,78925,2017-04-06T15:00:00+00:00,101830.0,-10.0,8,110,8.3,303.45,294.25,58,...,3,Le Lamentin,97213,CA du Centre de la Martinique,249720061,Martinique,972,Martinique,02,4
1,07110,2017-04-06T18:00:00+00:00,102910.0,-40.0,6,30,4.1,284.55,279.25,70,...,94,Guipavas,29075,Brest Métropole,242900314,Finistère,29,Bretagne,53,4
2,07481,2017-04-06T18:00:00+00:00,102170.0,80.0,3,340,6.6,285.75,275.55,50,...,235,Colombier-Saugnieu,69299,CC de l'Est Lyonnais (CCEL),246900575,Rhône,69,Auvergne-Rhône-Alpes,84,4
3,07661,2017-04-06T18:00:00+00:00,101610.0,110.0,3,200,3.2,286.85,284.75,87,...,115,Saint-Mandrier-sur-Mer,83153,Métropole Toulon-Provence-Méditerranée,248300543,Var,83,Provence-Alpes-Côte d'Azur,93,4
4,07761,2017-04-06T18:00:00+00:00,101540.0,100.0,3,170,1.2,288.65,282.35,66,...,5,Ajaccio,2a004,CA du Pays Ajaccien,242010056,Corse-du-Sud,2a,Corse,94,4
5,07130,2017-04-06T21:00:00+00:00,102900.0,80.0,2,360,3.5,282.25,279.35,82,...,36,Saint-Jacques-de-la-Lande,35281,Rennes Métropole,243500139,Ille-et-Vilaine,35,Bretagne,53,4
6,07591,2017-04-06T21:00:00+00:00,NaN,300.0,1,360,0.8,281.35,275.15,65,...,871,Embrun,05046,CC Serre-Ponçon,200067742,Hautes-Alpes,05,Provence-Alpes-Côte d'Azur,93,4
7,61980,2017-04-06T21:00:00+00:00,101620.0,-50.0,8,110,4.7,298.35,292.75,71,...,8,Sainte-Marie,97418,CA Intercommunale du Nord de la Réunion (CINOR),249740119,La Réunion,974,La Réunion,04,4
8,61976,2017-04-07T00:00:00+00:00,101250.0,10.0,None,110,7.4,300.15,295.15,74,...,7,None,None,None,None,None,None,None,None,4
9,07280,2017-04-07T03:00:00+00:00,102600.0,-70.0,6,360,3.6,277.35,273.65,77,...,219,Ouges,21473,Dijon Métropole,242100410,Côte-d'Or,21,Bourgogne-Franche-Comté,27,4


In [27]:
df = requests.get(url_base).json()["results"]
df.columns

ConnectionError: HTTPSConnectionPool(host='public.opendatasoft.com', port=443): Max retries exceeded with url: /api/explore/v2.1/catalog/datasets/donnees-synop-essentielles-omm/records (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1224d73d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))